# Ground Zero

## Packages and Datasets

In [1]:
import timeit
start_time = timeit.default_timer()  #timestamp to calculate total runtime

import pandas as pd
import numpy as np

import researchpy as rp
import missingno as msno

import seaborn as sns
import matplotlib.pyplot as plt

import itertools
import scipy.stats as ss

import folium
from folium import plugins

from sklearn.model_selection import train_test_split
from sklearn import feature_selection
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
pd.set_option('display.max_columns', None)  # prevent column output trancation
sns.set()  # change plot styling from Matlab's 90s feel to today's Seaborn.

In [2]:
# File Directories
path_crashes = '~/repos/vision_zero_chicago_project/crashes.sample2020.csv'
path_vehicles = '~/repos/vision_zero_chicago_project/vehicles.sample.csv'
path_people = '~/repos/vision_zero_chicago_project/people.sample.csv'

# Columns that will be read as categorical
# The fromkeys() method returns a dictionary with the specified keys and the specified value
#categoricals_crashes = dict.fromkeys(
#    [5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 35],
#    'category')
#categoricals_vehicles = dict.fromkeys([5, 9, 10, 11, 13, 14, 15, 16, 17, 38],
#                                      'category')
#categoricals_people = dict.fromkeys(
#    [7, 8, 10, 12, 13, 14, 15, 17, 21, 22, 23, 27], 'category')

# Import samples
crashes = pd.read_csv(
    path_crashes,
    parse_dates=["CRASH_DATE", "CRASH_DATE_EST_I", "DATE_POLICE_NOTIFIED"],
    low_memory=False,
    dtype=object)

vehicles = pd.read_csv(path_vehicles,
                       parse_dates=["CRASH_DATE"],
                       low_memory=False,
                       dtype=object)

people = pd.read_csv(path_people,
                     parse_dates=["CRASH_DATE"],
                     low_memory=False,
                     dtype=object)

## Our Understanding

### Feature Engineering Process

[Source](https://en.wikipedia.org/wiki/Feature_engineering)

The feature engineering process is:
- Brainstorming or testing features;
- Deciding what features to create;
- Creating features;
- Checking how the features work with your model;
- **Improving your features if needed;**
- Go back to brainstorming/creating more features until the work is done.

### Some Terms

[Source](https://datascience.stackexchange.com/questions/29006/feature-selection-vs-feature-extraction-which-to-use-when)

**Feature Extraction and Feature Engineering:** transformation of raw data into features suitable for modeling; 

- Texts(ngrams, word2vec, tf-idf etc)
- Geospatial data(lat, long)
- Date and time(day, month, week, year, rolling based)
- Time series, web, etc
- Dimensional Reduction Techniques (PCA, SVD, Eigen-Faces etc)
- Clustering (DBSCAN etc)
- And others.

---

**Feature / Data Transformation:** transformation of data to improve the accuracy of the algorithm;
- Normalization and changing distribution(Scaling)
- Interactions
- Handling missing Values
- Handling outliers
- Handling time-date / mixed variables
- Encoding categorical features
- Variable types 
- And others.

---

**Feature Selection:** removing unnecessary features.

- Statistical approaches
- Selection by modeling
- Grid search
- Cross Validation
- And others

---

For the purpose of workflow, we will split **Feature Selection** into; 
- **feature reduction** to remove unnecessary features, 
- and **feature selection** to select features by their importance scores through ML/statistical techniques.

# Data Merge & Split

## Merging datasets

The goal is to successfully merge all datasets by increasing crash records as little as possible.

In [3]:
print(crashes.shape)
print(vehicles.shape)
print(people.shape)

(33408, 49)
(75673, 72)
(82049, 30)


**Data Merge Strategy**

- Filter out passengers from **`people`**, and left-join it to **`vehicles`** on `CRASH_RECORD_ID`, `RD_NO`, `CRASH_DATE`, `VEHICLE_ID`.

- Then inner-join this dataset to **`crashes`** on `CRASH_RECORD_ID`, `RD_NO`, `CRASH_DATE`.

---

**Reasoning**

- `people` has **many-to-one relationship** with `vehicles` but a pedestrian in `vehicles` has **one-to-one** with `people`,

- Passengers don't have <u>a trajectory separate from the unit</u> they belong to and they don't contribute to accidents with their existence but they are involved if they got harmed through `injuries` features,

- This method increases `crashes` records **only per active parties involved**.

In [4]:
# Filter only non-pessenger people (drivers, pedestrian, cyclist, etc.)
non_passengers = people[people.PERSON_ID.str.contains('^O')]

In [5]:
# Left join non-peseenger people with vehicles
vehicles_with_people = vehicles.merge(non_passengers, 
                                      how='left', 
                                      on=['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'VEHICLE_ID'])
vehicles_with_people.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,TOWED_I,FIRE_I,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO,ILCC_NO,COMMERCIAL_SRC,GVWR,CARRIER_NAME,CARRIER_STATE,CARRIER_CITY,HAZMAT_PLACARDS_I,HAZMAT_NAME,UN_NO,HAZMAT_PRESENT_I,HAZMAT_REPORT_I,HAZMAT_REPORT_NO,MCS_REPORT_I,MCS_REPORT_NO,HAZMAT_VIO_CAUSE_CRASH_I,MCS_VIO_CAUSE_CRASH_I,IDOT_PERMIT_NO,WIDE_LOAD_I,TRAILER1_WIDTH,TRAILER2_WIDTH,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS,PERSON_ID,PERSON_TYPE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,HOSPITAL,EMS_AGENCY,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,228894,2524b06fc6634bd349fc1d7f132c8362529473768f6ac2...,JA453603,2017-10-01 02:20:00,1,DRIVER,NaN,224185,NaN,DODGE,CHARGER,IL,2012,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,Y,NaN,1,NaN,PRIVATE TOW,UNKNOWN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O228894,DRIVER,NaN,KANKAKKE,IL,60901,M,18,IL,D,SAFETY BELT USED,"DEPLOYED, FRONT",NONE,NONINCAPACITATING INJURY,NaN,NaN,NaN,UNKNOWN,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,228895,2524b06fc6634bd349fc1d7f132c8362529473768f6ac2...,JA453603,2017-10-01 02:20:00,2,PARKED,NaN,224187,NaN,GENERAL MOTORS CORP.,YUKON,IL,2003,UNKNOWN,PASSENGER,PERSONAL,N,PARKED,Y,NaN,0,NaN,PRIVATE TOW,UNKNOWN,N,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,228896,2524b06fc6634bd349fc1d7f132c8362529473768f6ac2...,JA453603,2017-10-01 02:20:00,3,PARKED,NaN,224191,NaN,SATURN,UNKNOWN,IL,2000,UNKNOWN,PASSENGER,PERSONAL,N,PARKED,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR-RIGHT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,228905,f7210aec9ac43a7b36c30b49033adebcb3060b96580341...,JA453640,2017-10-01 02:41:00,1,DRIVER,NaN,224199,NaN,FORD,WINDSTAR,IL,2007,NONE,VAN/MINI-VAN,PERSONAL,E,STRAIGHT AHEAD,Y,NaN,1,NaN,NaN,NaN,NaN,Y,Y,Y,Y,NaN,N,N,N,NaN,NaN,Y,NaN,NaN,TOTAL (ALL AREAS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O228905,DRIVER,NaN,CHICAGO,IL,60632,M,43,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,IMPROPER LANE CHANGE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,228906,f7210aec9ac43a7b36c30b49033adebcb3060b96580341...,JA453640,2017-10-01 02:41:00,2,DRIVERLESS,NaN,224203,NaN,CHEVROLET,CRUZE,IL,2012,NONE,PASSENGER,NOT IN USE,UNKNOWN,PARKED,Y,NaN,0,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,Y,Y,NaN,NaN,Y,NaN,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(vehicles_with_people.shape)

(75838, 98)


In [7]:
#wtf = df_test.groupby('CRASH_RECORD_ID')
#wtf.head(n=20)
# should we filter as 'OCCUPANT_CNT >= 1' ?
# OCCUPANT_CNT = The number of people in the unit, as determined by the reporting officer.

In [8]:
merged_data = crashes.merge(vehicles_with_people, 
                            how='inner', 
                            on=['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE'])

print('Dimensions of the data after merging', merged_data.shape)

Dimensions of the data after merging (68503, 144)


**Inspection 1**

In [9]:
merged_data.groupby('CRASH_RECORD_ID').head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,CRASH_UNIT_ID,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,TOWED_I,FIRE_I,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO,ILCC_NO,COMMERCIAL_SRC,GVWR,CARRIER_NAME,CARRIER_STATE,CARRIER_CITY,HAZMAT_PLACARDS_I,HAZMAT_NAME,UN_NO,HAZMAT_PRESENT_I,HAZMAT_REPORT_I,HAZMAT_REPORT_NO,MCS_REPORT_I,MCS_REPORT_NO,HAZMAT_VIO_CAUSE_CRASH_I,MCS_VIO_CAUSE_CRASH_I,IDOT_PERMIT_NO,WIDE_LOAD_I,TRAILER1_WIDTH,TRAILER2_WIDTH,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS,PERSON_ID,PERSON_TYPE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,HOSPITAL,EMS_AGENCY,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,NaN,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2019-06-11 09:05:00,UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794778764,-87.623828038,POINT (-87.623828038036 41.794778764028),667550,1,DRIVER,NaN,635502,NaN,CHEVROLET,EQUINOX,IL,2017,UNKNOWN,SPORT UTILITY VEHICLE (SUV),UNKNOWN/NA,W,STRAIGHT AHEAD,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O667550,DRIVER,NaN,HOMEWOOD,IL,60430,F,56,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,NaN,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2019-06-11 09:05:00,UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794778764,-87.623828038,POINT (-87.623828038036 41.794778764028),667551,2,DRIVER,1,635496,NaN,CHEVROLET,SILVERADO,IL,2016,NONE,PICKUP,PERSONAL,W,STARTING IN TRAFFIC,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O667551,DRIVER,NaN,PALOS HILLS,IL,60465,M,51,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST 

**Inspection 2**

In [10]:
merged_data[merged_data.RD_NO.eq('JC301403')] #Wrong joins could result in 6 rows

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,CRASH_UNIT_ID,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,TOWED_I,FIRE_I,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO,ILCC_NO,COMMERCIAL_SRC,GVWR,CARRIER_NAME,CARRIER_STATE,CARRIER_CITY,HAZMAT_PLACARDS_I,HAZMAT_NAME,UN_NO,HAZMAT_PRESENT_I,HAZMAT_REPORT_I,HAZMAT_REPORT_NO,MCS_REPORT_I,MCS_REPORT_NO,HAZMAT_VIO_CAUSE_CRASH_I,MCS_VIO_CAUSE_CRASH_I,IDOT_PERMIT_NO,WIDE_LOAD_I,TRAILER1_WIDTH,TRAILER2_WIDTH,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS,PERSON_ID,PERSON_TYPE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,HOSPITAL,EMS_AGENCY,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,NaN,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2019-06-11 09:05:00,UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794778764,-87.623828038,POINT (-87.623828038036 41.794778764028),667550,1,DRIVER,NaN,635502,NaN,CHEVROLET,EQUINOX,IL,2017,UNKNOWN,SPORT UTILITY VEHICLE (SUV),UNKNOWN/NA,W,STRAIGHT AHEAD,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O667550,DRIVER,NaN,HOMEWOOD,IL,60430,F,56,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,NaN,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2019-06-11 09:05:00,UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794778764,-87.623828038,POINT (-87.623828038036 41.794778764028),667551,2,DRIVER,1,635496,NaN,CHEVROLET,SILVERADO,IL,2016,NONE,PICKUP,PERSONAL,W,STARTING IN TRAFFIC,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O667551,DRIVER,NaN,PALOS HILLS,IL,60465,M,51,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST 

## Splitting the data

In [11]:
# Alternative: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
raw_train, raw_test = train_test_split(merged_data, test_size=0.33, random_state=42, shuffle=True)

In [12]:
print(raw_train.shape)
print(raw_test.shape)

(45897, 144)
(22606, 144)


**Write datasets as csv**

In [13]:
# save the train and test file
#train.to_csv('~/repos/vision_zero_chicago_project/sprint 2/raw_train.csv')
#test.to_csv('~/repos/vision_zero_chicago_project/sprint 2/raw_test.csv')

# Feature Reduction

## Remove Features with Missing Values

### Remove Dummy Features

Removing **49** dummy features that are **heavily missing** and/or **have no explanation** in the data dictionary provided by City of Chicago.
- 47 features are from `AREA_00_I` to `HAZMAT_CLASS` except `FIRST_CONTACT_POINT`. 
- `CMRC_VEH_I` and `FIRE_I`.

In [14]:
# Get indicies and inspect these columns
for c in 'AREA_00_I', 'FIRST_CONTACT_POINT', 'HAZMAT_CLASS':
    print(raw_train.columns.get_loc(c))

raw_train.iloc[:, 70:118]

70
84
117


,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO,ILCC_NO,COMMERCIAL_SRC,GVWR,CARRIER_NAME,CARRIER_STATE,CARRIER_CITY,HAZMAT_PLACARDS_I,HAZMAT_NAME,UN_NO,HAZMAT_PRESENT_I,HAZMAT_REPORT_I,HAZMAT_REPORT_NO,MCS_REPORT_I,MCS_REPORT_NO,HAZMAT_VIO_CAUSE_CRASH_I,MCS_VIO_CAUSE_CRASH_I,IDOT_PERMIT_NO,WIDE_LOAD_I,TRAILER1_WIDTH,TRAILER2_WIDTH,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
15125,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57881,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,FRONT-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,TOTAL (ALL AREAS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53511,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIDE-RIGHT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6265,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,NaN,TOTAL (ALL AREAS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
860,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


95 features are remained after removing 49 features above.

In [15]:
# Drop those features except 'FIRST_CONTACT_POINT'
raw_train = raw_train.drop(raw_train.loc[:,'AREA_00_I':'AREA_99_I'], axis=1)
raw_train = raw_train.drop(raw_train.loc[:, 'CMV_ID':'HAZMAT_CLASS'], axis=1)
raw_train = raw_train.drop(raw_train.loc[:, ['CMRC_VEH_I', 'FIRE_I']], axis=1)
raw_train.shape

### Remove Completely Missing Features

Removing additional two features that are completely missing.

In [17]:
print(sum(raw_train.isnull().mean() == 1))
print(raw_train.columns[raw_train.isnull().mean() == 1].tolist())

2
['SEAT_NO', 'EMS_RUN_NO']


93 features are remained.

In [18]:
raw_train = raw_train.drop(raw_train.loc[:, ['SEAT_NO', 'EMS_RUN_NO']], axis=1)
raw_train.shape

(45897, 93)

### Inspect Features Missing > 50%

Still, there are 23 features that have missing values more than 50%!

In [19]:
# Subset the data to include the columns only with nulls in the output
raw_null_above50 = raw_train[raw_train.columns[raw_train.isnull().mean().round(2) > 0.5].tolist()]
print('\033[1m', 'These', len(raw_null_above50.columns),
      'columns have missing values more than 50%!', '\033[0m')
raw_null_above50.isnull().mean().round(3)

 These 23 columns have missing values more than 50%! 


CRASH_DATE_EST_I          0.930
LANE_CNT                  0.670
INTERSECTION_RELATED_I    0.756
NOT_RIGHT_OF_WAY_I        0.959
HIT_AND_RUN_I             0.702
PHOTOS_TAKEN_I            0.987
STATEMENTS_TAKEN_I        0.977
DOORING_I                 0.997
WORK_ZONE_I               0.994
WORK_ZONE_TYPE            0.996
WORKERS_PRESENT_I         0.999
NUM_PASSENGERS            0.845
TOWED_I                   0.870
EXCEED_SPEED_LIMIT_I      0.997
TOWED_BY                  0.905
TOWED_TO                  0.940
HOSPITAL                  0.837
EMS_AGENCY                0.899
PEDPEDAL_ACTION           0.976
PEDPEDAL_VISIBILITY       0.976
PEDPEDAL_LOCATION         0.976
BAC_RESULT VALUE          0.998
CELL_PHONE_USE            0.999
dtype: float64

<a id='2.1.3.1'></a>
#### Features that can be imputed/encoded

**Features that can be imputed with No**

*Jump to [3.3.1](#3.3.1) for their imputation*

1. `CRASH_DATE_EST_I`: Crash date estimated by desk officer or reporting party (only used in cases where crash is reported at police station days after the crash).


2. `INTERSECTION_RELATED_I`: A field observation by the police officer whether an intersection played a role in the crash. Does not represent whether or not the crash occurred within the intersection.


3. `NOT_RIGHT_OF_WAY_I`: Whether the crash begun or first contact was made outside of the public right-of-way.


4. `HIT_AND_RUN_I`: Crash did/did not involve a driver who caused the crash and fled the scene without exchanging information and/or rendering aid.


5. `CELL_PHONE_USE`: Whether person was/was not using cellphone at the time of the crash, as determined by the reporting officer.


6. `EXCEED_SPEED_LIMIT_I`: Indicator of whether the unit was speeding, as determined by the reporting officer.


7. `DOORING_I`: Whether crash involved a motor vehicle occupant opening a door into the travel path of a bicyclist, causing a crash.


8. `WORK_ZONE_I`: Whether the crash occurred in an active work zone.


9. `WORKERS_PRESENT_I`: Whether construction workers were present in an active work zone at crash location.


10. `BAC_RESULT VALUE`: Driver’s blood alcohol concentration test result (fatal crashes may include pedestrian or cyclist results). This feature can be engineered as **`alcohol_consumption` = yes/no**.

In [20]:
print(raw_train.CRASH_DATE_EST_I.value_counts())
print()
print(raw_train.INTERSECTION_RELATED_I.value_counts())
print()
print(raw_train.NOT_RIGHT_OF_WAY_I.value_counts())
print()
print(raw_train.HIT_AND_RUN_I.value_counts())
print()
print(raw_train.CELL_PHONE_USE.value_counts())
print()
print(raw_train.EXCEED_SPEED_LIMIT_I.value_counts())
print()
print(raw_train.DOORING_I.value_counts())
print()
print(raw_train.WORK_ZONE_I.value_counts())
print()
print(raw_train.WORKERS_PRESENT_I.value_counts())
print()
print(raw_train['BAC_RESULT VALUE'].value_counts())

Y    2730
N     469
Name: CRASH_DATE_EST_I, dtype: int64

Y    10659
N      520
Name: INTERSECTION_RELATED_I, dtype: int64

Y    1725
N     175
Name: NOT_RIGHT_OF_WAY_I, dtype: int64

Y    13063
N      592
Name: HIT_AND_RUN_I, dtype: int64

Y    36
N    13
Name: CELL_PHONE_USE, dtype: int64

Y    108
N     32
Name: EXCEED_SPEED_LIMIT_I, dtype: int64

Y    105
N     55
Name: DOORING_I, dtype: int64

Y    205
N     60
Name: WORK_ZONE_I, dtype: int64

Y    55
N     3
Name: WORKERS_PRESENT_I, dtype: int64

0.2100    10
0.0000     8
0.1400     8
0.2000     7
0.1100     7
0.1700     7
0.1500     5
0.1200     5
0.1900     5
0.2200     3
0.1000     3
0.1800     3
0.1300     3
0.2700     2
0.1600     2
0.2300     2
0.2500     2
0.4400     1
0.3300     1
0.3100     1
0.2800     1
0.2400     1
0.3000     1
0.3200     1
0.0300     1
0.2600     1
0.3500     1
0.0100     1
Name: BAC_RESULT VALUE, dtype: int64


**Features can be imputed with 0**

11. `NUM_PASSENGERS`: Number of passengers in the vehicle. The driver **is not** included. More information on passengers is in the People dataset.

In [21]:
print(raw_train.NUM_PASSENGERS.value_counts())

1     4964
2     1353
3      535
4      162
5       55
6       20
7        6
8        4
10       3
9        3
38       1
12       1
16       1
19       1
Name: NUM_PASSENGERS, dtype: int64


#### Features that are not meaningful

7 non-meaningful features below are removed. 86 features are remained. 

12. `PHOTOS_TAKEN_I`: Whether the Chicago Police Department took photos at the location of the crash.
13. `STATEMENTS_TAKEN_I`: Whether statements were taken from unit(s) involved in crash.
14. `TOWED_I`: Indicator of whether the vehicle was towed.
15. `TOWED_BY`: Entity that towed the unit, if relevant.
16. `TOWED_TO`: Location to which the unit was towed, if relevant.
17. `EMS_AGENCY`: EMS agency who transported person injured in crash to the hospital.
18. `WORK_ZONE_TYPE`: The type of work zone, if any.

In [22]:
raw_train = raw_train.drop(raw_train.loc[:, [
    'PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I','TOWED_I','TOWED_BY','TOWED_TO','EMS_AGENCY','WORK_ZONE_TYPE'
]], 
                           axis=1)
raw_train.shape

(45897, 86)

#### Features that waits a decision

19. `LANE_CNT`: Total number of through lanes in either direction, excluding turn lanes, as determined by reporting officer (0 = intersection).
20. `HOSPITAL`: Hospital to which person injured in the crash was taken.
21. `PEDPEDAL_ACTION`: Action of pedestrian or cyclist at the time of crash.
22. `PEDPEDAL_VISIBILITY`: Visibility of pedestrian of cyclist safety equipment in use at time of crash.
23. `PEDPEDAL_LOCATION`: Location of pedestrian or cyclist at the time of crash.

In [25]:
raw_train.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,DOORING_I,WORK_ZONE_I,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,CRASH_UNIT_ID,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,FIRST_CONTACT_POINT,PERSON_ID,PERSON_TYPE,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,HOSPITAL,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
15125,23ad95c12c156fb65161003179600144c3a4756fc53215...,JD268602,NaN,2020-06-18 22:06:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,UNKNOWN,NOT ON SCENE (DESK REPORT),INJURY AND / OR TOW DUE TO CRASH,Y,NaN,NaN,"OVER $1,500",2020-06-18 22:30:00,FAILING TO YIELD RIGHT-OF-WAY,UNABLE TO DETERMINE,1200,N,CALIFORNIA AVE,1423,NaN,NaN,NaN,2,"REPORTED, NOT EVIDENT",5,0,0,0,5,0,0,22,5,6,41.902969953,-87.697027812,POINT (-87.697027812314 41.902969952568),902018,2,DRIVER,2,855303,TOYOTA,4RUNNER,IL,1996,UNKNOWN,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,3,NaN,OTHER,O902018,DRIVER,CHICAGO,IL,60651,F,53,IL,D,USAGE UNKNOWN,NOT APPLICABLE,NONE,"REPORTED, NOT EVIDENT",NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
57881,d3e15b2edf3c052ccd3baecd774b8828a05cb304628de8...,JC185553,NaN,2019-03-14 17:14:00,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLOUDY/OVERCAST,DAWN,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),4,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,NaN,NaN,"OVER $1,500",2019-03-14 17:16:00,FAILING TO YIELD RIGHT-OF-WAY,UNABLE TO DETERMINE,5200,S,LAKE PARK AVE,234,NaN,NaN,NaN,2,NONINCAPACITATING INJURY,2,0,0,2,0,1,0,17,5,3,41.800941976,-87.587636792,POINT (-87.587636792157 41.800941975966),603327,1,DRIVER,NaN,575025,INFINITI,FX35,IL,2012,NONE,PASSENGER,PERSONAL,SE,ENTERING TRAFFIC LANE FROM PARKING,1,NaN,FRONT-LEFT,O603327,DRIVER,CHICAGO,IL,60615,F,37,IL,DM,NONE PRESENT,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,FAILED TO YIELD,PARKED VEHICLES,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2428,b25aee5a038a0f9e5da4748b1ed592b228718b6ed744d3...,JD335607,NaN,2020-08-17 13:00:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",2020-08-17 14:50:00,UNABLE TO DETERMINE,UNABLE TO DETERMINE,799,N,SEDGWICK ST,1831,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,13,2,8,41.896501897,-87.638466532,POINT (-87.638466531596 41.896501896875),940304,2,DRIVER,NaN,891380,TESLA,OTHER (EXPLAIN IN NARRATIVE),IL,2020,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1,NaN,TOTAL (ALL AREAS),O940304,DRIVER,CHICAGO,IL,60654,M,36,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
53511,c14bbcab3cdf958304f35ba3f19f23613259124a8fba08...,JB124218,NaN,2018-01-21 13:45:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),3,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",2018-01-21 13:50:00,FOLLOWING TOO CLOSELY,NOT APPLICABLE,6,E,95TH ST,634

## Remove Features with Low Variance

Remove all features whose variance do not meet some threshold. By default `feature_selection.VarianceThreshold()` removes features with zero variance or features that have the same value for all samples.

In [ ]:
# Numeric data
raw_train.select_dtypes('number').columns

# Remove features with zero variance
sel = feature_selection.VarianceThreshold()
train_variance = sel.fit_transform(numeric_set)
train_variance.shape

## Remove Highly Correlated Features

Check and fix collinearity to prevent over-fitting.

Two options
- **Variation Inflation Factor** that checks for estimated coefficients that are inflated when multicollinearity exists. 
- Check numerical correlation (after encoding)?

**Variation Inflation Factor (VIF)**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=100):
cols = X.columns
variables = np.arange(X.shape[1])
dropped=True
while dropped:
    dropped=False
    c = X[cols[variables]].values
    vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

    maxloc = vif.index(max(vif))
    if max(vif) > thresh:
        print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
        variables = np.delete(variables, maxloc)
        dropped=True

print('Remaining variables:')
print(X.columns[variables])
return X[cols[variables]]

**Checking numerical correlation**

After encoding ?

In [ ]:
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features
df.drop(df[to_drop], axis=1)

# Feature Transformation

## Feature Types

Decide the correct data type for each feature.

## Handling Outliers 

### Categorical Outliers

Example 1: A, B, C, X.

Example 2: Apple, Orange, Pear or Blueberry, Raspberry, Strawberry, Grape.

"*The fact is **no outlier** exists in the examples above, for an outlier to exist there must be **a measure of distance**. This is intrinsic to numeric data types because there is a Euclidean distance between numbers.*"

**Categorical Outliers don’t exist without a context.**

> "*Techniques like **string indexing** or **one-hot encoding** on a categorical value to convert it from a string value to a numeric value and then use k-means clustering to find outliers.*"

> "*the numeric vector assigned to the String value during the one hot encoding practice still lacks **the domain context**. It does satisfy the input constraint of the k-means model and the model will produce an output. However the output will not select the appropriate outlier, as the input **was not truly valid**.*"

> "*There is an alternative. **K-modes** is designed to handle categorical values without the need for String Indexing or One Hot Encoding.*"

### Numerical Outliers

Numerical features to consider
- `POSTED_SPEED_LIMIT`
- `LANE_CNT`
- `BEAT_OF_OCCURRENCE`
- `NUM_UNITS`
- `INJURIES_TOTAL`
- `INJURIES_FATAL`
- `INJURIES_INCAPACITATING` and other injuries.
- `CRASH_HOUR`
- `CRASH_DAY_OF_WEEK`
- `CRASH_MONTH`
- `LATITUDE`
- `LONGITUDE`
- `UNIT_NO`
- `NUM_PASSENGERS`
- `VEHICLE_YEAR`
- `OCCUPANT_CNT`
- `AGE`

In [40]:
numericals=['POSTED_SPEED_LIMIT', 'LANE_CNT', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'INJURIES_TOTAL', 
            'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 
            'LATITUDE', 'LONGITUDE', 'UNIT_NO', 'NUM_PASSENGERS', 'VEHICLE_YEAR', 'OCCUPANT_CNT', 'AGE']

for col in numericals:
    print(raw_train[col].value_counts())
    print()

30    33929
35     3390
25     2917
20     1753
15     1459
10      856
0       505
40      468
5       312
45      225
55       42
50       10
3         7
1         6
9         4
60        3
33        2
11        2
2         2
4         1
65        1
70        1
39        1
24        1
Name: POSTED_SPEED_LIMIT, dtype: int64

2     7135
4     3686
1     2503
3      675
0      459
6      343
5      177
8      119
7       14
10      13
12       4
9        3
99       2
11       1
Name: LANE_CNT, dtype: int64

1834    535
114     490
815     468
813     462
1831    445
       ... 
422      34
1125     30
1653     18
1652     14
1655     11
Name: BEAT_OF_OCCURRENCE, Length: 274, dtype: int64

2     38841
3      4072
1      1358
4      1114
5       264
6       121
7        76
8        21
10        9
12        8
9         7
11        6
Name: NUM_UNITS, dtype: int64

0    38689
1     5231
2     1227
3      391
4      149
5       68
7       21
6       20
8        4
Name: INJURIES_TOTAL, dtype: 

**`POSTED_SPEED_LIMIT`**

**ACTION!!!: Replace anything below 15 with 15???**

[Automated Speed Enforcement FAQ](https://www.chicago.gov/city/en/depts/cdot/supp_info/children_s_safetyzoneporgramautomaticspeedenforcement/automated_speed_enforcementfrequentlyaskedquestions.html)

What are the Safety Zone times and speed limits?

**School Zones:**

School days, Monday through Friday
- 7 am to 4 pm: **20 mph** speed limit when children are present in the safety zone.

- 7 am to 7:00 pm: The posted speed limit, **normally 30 mph**. Each Child Safety Zone has the posted speed limit clearly displayed. Motorists’ should always follow the posted speed limit in each Child Safety Zone. 

**Park Zones:**

365 Days a Year
- Times are variable. Visit www.chicago.gov for more information on specific parks. Generally, Park Zones will be enforced from 6 a.m. to 11 p.m.: enforcing the posted speed limit, **normally 30 mph**. 

**`LANE_CNT`**

- 3 rows of 9
- 13 rows of 10
- 1 row of 11
- 4 rows of 12
- 2 rows of 99

##  Handling Missing Values

<a id='3.3.1'></a>

###  Imputation of binary features in 2.1.3.1

*[Back to 2.1.3.1](#2.1.3.1)*

In [ ]:
binary_columns = ['CRASH_DATE_EST_I', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 
                  'CELL_PHONE_USE', 'EXCEED_SPEED_LIMIT_I', 'DOORING_I', 'WORK_ZONE_I', 'WORKERS_PRESENT_I']
for col in binary_columns:
    raw_train[col] = raw_train[col].fillna('N')

In [ ]:
raw_train[binary_columns].isna().any()

In [ ]:
#raw_train['BAC_RESULT VALUE']
#**`alcohol_consumption` = yes/no**.

###  Remove rows with missing data

In [ ]:
# Subset the data to include the columns only with nulls in the output
raw_null_below50 = raw_train[raw_train.columns[raw_train.isnull().mean().round(2) < 0.5].tolist()]
print('\033[1m', 'These', len(raw_null_above50.columns),
      'columns have missing values less than 50%!', '\033[0m')
raw_null_below50.isnull().mean().round(3)b

###  Scikit-learn's Imputer Class

## Encoding Categorical Features 

Number of unique values per column.

In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

unique_counts = raw_train.from_records(
    [(col, raw_train[col].nunique()) for col in raw_train.columns],
    columns=['column_name', 'num_unique']).sort_values(by=['num_unique'])

#unique_counts[Numm_]

## Feature Transformation

- Log, Box-cox transformation etc.

## Feature Scaling

## Handling Time-Date & Mixed Variables


# Feature Engineering

[Source](https://en.wikipedia.org/wiki/Feature_engineering)

The feature engineering process is:
- **Brainstorming** or **testing** features;
- **Deciding** what features to create;
- **Creating** features;
- **Checking** how the features work with your model;
- **Improving** your features if needed;
- **Go back** to brainstorming/creating more features until the work is done.

## Target Features

Pick **`INJURIES_TOTAL`** as a numerical target and/or encode **`INJURIES_FATAL`** as a categorical target.

- `INJURIES_TOTAL`: Total persons sustaining **fatal, incapacitating, non-incapacitating, and possible injuries** as determined by the reporting officer.


- `INJURIES_FATAL`: Total persons sustaining fatal injuries in the crash.


- `INJURIES_INCAPACITATING`: Total persons sustaining **incapacitating/serious injuries** in the crash as determined by the reporting officer. Any injury other than fatal injury, which prevents the injured person from walking, driving, or normally continuing the activities they were capable of performing before the injury occurred. <u>Includes severe lacerations, broken limbs, skull or chest injuries, and abdominal injuries</u>.


- `INJURIES_NON_INCAPACITATING`: Total persons sustaining non-incapacitating injuries in the crash as determined by the reporting officer. Any injury, **other than fatal or incapacitating injury**, which is evident to observers at the scene of the crash. <u>Includes lump on head, abrasions, bruises, and minor lacerations</u>.


- `INJURIES_REPORTED_NOT_EVIDENT`: Total persons sustaining **possible injuries** in the crash as determined by the reporting officer. <u>Includes momentary unconsciousness, claims of injuries not evident, limping, complaint of pain, nausea, and hysteria</u>.


- ~`INJURIES_NO_INDICATION`: Total persons sustaining **no injuries** in the crash as determined by the reporting officer.~


- ~`INJURIES_UNKNOWN`: Total persons for whom injuries sustained, if any, are unknown.~

In [41]:
target_features = ['INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 
                   'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN']

for col in target_features:
    print(raw_train[col].value_counts())
    print()

0    38689
1     5231
2     1227
3      391
4      149
5       68
7       21
6       20
8        4
Name: INJURIES_TOTAL, dtype: int64

0    45744
1       53
2        3
Name: INJURIES_FATAL, dtype: int64

0    44825
1      818
2       97
3       37
5       15
4        8
Name: INJURIES_INCAPACITATING, dtype: int64

0    41635
1     3184
2      667
3      214
4       73
5       14
6        8
7        3
8        2
Name: INJURIES_NON_INCAPACITATING, dtype: int64

0    43412
1     1856
2      366
3       99
4       33
5       27
6        4
7        3
Name: INJURIES_REPORTED_NOT_EVIDENT, dtype: int64

2     20636
1     13529
3      6488
4      2398
5      1045
0       872
6       470
7       176
8        91
9        34
10       25
11       12
12        5
13        5
15        4
18        2
40        2
14        2
21        2
31        1
46        1
Name: INJURIES_NO_INDICATION, dtype: int64

0    45800
Name: INJURIES_UNKNOWN, dtype: int64



## Predictors Features

# Feature Selection

## Univariate Feature Selection

Uses univariate statistical tests to select features having the highest correlation to the target. 

In [ ]:
#from sklearn.feature_selection import SelectKBest, f_classif

# Select 50 features
k_best = SelectKBest(score_func=f_classif, k=50)

# Fit on train set
fit = k_best.fit(X_train, y_train)

# Transform train set
univariate_features = fit.transform(X_train)

## Recursive Feature Elimination

Eliminate features recursively by importance until the specified number of features is reached. It can be used with any model that assigns weights to features, either through `coef_` or `feature_importances_`.

In [ ]:
#from sklearn.feature_selection import RFE

# Select 50 features
rfe = RFE(rfc, n_features_to_select=50)

# Fit on train set
fit = rfe.fit(X_train, y_train)

# Transform train set
recursive_features = fit.transform(X_train)

## Dimensionality Reduction

Dimensionality reduction on any feature if necessary by 
- transform features into a lower dimension (PCA),
- simple arithmetics and KPIs,
- other methods.

# Baseline Models

## Baseline Models


Fundamentally, a baseline is a model that is both simple to set up and has a reasonable chance of providing decent results.

Decision tree models perform well on imbalanced data.

**Model 1**

In [ ]:
# prepare for modeling
X_train_df = train.drop(['id', 'target'], axis=1)
y_train = train['target']

# scaling data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_df)
lr = LogisticRegression(solver='liblinear')
lr_scores = cross_val_score(lr,
                            X_train,
                            y_train,
                            cv=5,
                            scoring='roc_auc')
print('LR Scores: ', lr_scores)

**Model 2**

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)

# predict
rfc_predict = rfc.predict(x_test)# check performance

print('ROCAUC score:',roc_auc_score(y_test, rfc_predict))
print('Accuracy score:',accuracy_score(y_test, rfc_predict))
print('F1 score:',f1_score(y_test, rfc_predict))

## Success Measures

[Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix)

Accuracy is not suitable to use with imbalanced data.

- **Precision:** the number of true positives divided by all positive predictions. Precision is also called Positive Predictive Value. It is a measure of a classifier’s exactness. Low precision indicates a high number of false positives.


- **Recall:** the number of true positives divided by the number of positive values in the test data. The recall is also called **Sensitivity** or the **True Positive Rate (TPR)**. It is a measure of a classifier’s completeness. Low recall indicates a high number of false negatives.


- **F1:** Score: the weighted <u>average of precision and recall</u>.


- **Area Under ROC Curve** 

## Next Steps

[Source](https://blog.insightdatascience.com/always-start-with-a-stupid-model-no-exceptions-3a22314b9aaa)

Get a quick performance benchmark with baseline model, then inspect what this model is struggling with (errors) can help you choose a next approach.
- A baseline helps you understand your data
- A baseline helps you understand your task

### Handling Class Imbalance

- Resampling techniques (Sub-sampling)
- Penalize algorithms (Cost-Sensitive Training)
- Change performance metric
- Change algorithms